In [ ]:
# Checking core SDK version
import azureml.core 
print('SDK version: ', azureml.core.VERSION)

In [ ]:
#Initialising workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

In [ ]:
# Creating an experiment
from azureml.core import Experiment
experiment_name = 'train-on-amlcompute'
experiment = Experiment(workspace=ws, name=experiment_name)

In [ ]:
# Checking supported amlcompute in my region(west europe)
from azureml.core.compute import ComputeTarget, AmlCompute

AmlCompute.supported_vmsizes(workspace=ws)

In [ ]:
# Creating a project repository
import os
import shutil

project_folder = './train-on-amlcompute'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train.py', project_folder)

In [ ]:
# Create docker based scikit-learn Environment
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("myenv")

myenv.docker.enabled = True
myenv.python.conda_dependencies = CondaDependencies.create(conda_packages=('scikit-learn'))

In [ ]:
# Provision as a persistent compute target(Basic)

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "mlcompute"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
    

In [ ]:
# Configure and Run a Script
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

src = ScriptRunConfig(source_directory=project_folder, script='train.py')

#set target
src.run_config.target = cpu_cluster_name

# set environment
src.run_config.environment = myenv

run = experiment.submit(config=src)
run

In [ ]:
# Get status of amlcompute target
cpu_cluster.get_status().serialize()